In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
import scipy.signal as signal
from joblib import Parallel, delayed

In [ ]:
# Simulation function
def simulate(kk):
    matplotlib.use('agg')
    SNR = 80
    Eo = .4
    alpha= .2
    tauv = 30
    tauo = 2
    fs = 100
    cutOffFreq = np.linspace(0.001,.7999,10)
    # Stimulus Shape. Should be a 6 second stimulus
    traps = np.convolve(np.ones(int(2*fs)), np.ones(int(5*fs)))
    win = signal.hamming(len(traps))
    traps = traps/traps.max()*1#*win
    u=np.zeros(int(fs*100*10))
    tauv = tauv + np.zeros(len(u))
    signs = [0,0]
    # Make 20 stimuli of one condition immediately followed by another condition
    # which causes an alternate directionality of connectivity
    for i in range(20):
        if signs[0] >= 10 or signs[1] >= 10:
           s = np.argmin(signs)*2-1 
        else:
            s =  np.sign(np.random.randn())
            if s <0:
                signs[0] += 1
            else:
                signs[1] +=1
        u[int(fs*10)+i*fs*50:int(fs*10)+i*fs*50 + len(traps)] += s*traps
        u[int(2*fs*10)+i*fs*50:int(2*fs*10)+i*fs*50 + len(traps)] += -s*traps
    X = np.zeros((len(u),100))
    q = np.zeros((len(u),100))
    p = np.zeros((len(u),100))
    v = np.zeros((len(u),100))
    f = np.zeros((len(u),100))
    s = np.zeros((len(u),100))
    high = .1
    low = .01
    rHR = (high+low)*np.random.rand(len(u)) +low
    h = 1/fs
    np.random.seed(kk)
    # Bilinear neural state matrices
    A1 = .4*(np.random.randn(50,50))
    A2 = .4*(np.random.randn(50,50))
    A3 = .4*(np.random.randn(50,50))
    A4 = .4*(np.random.randn(50,50))
    A = np.eye(100)
    A[:50,:50] = A1
    A[50:,50:] = A3
    C = .1*np.random.randn(100,100)
    C2 = .1*np.random.randn(100,100)
    U2 = 10*np.random.binomial(1,.3,(len(u),100)).astype(float)
    U2[:,25:75] *= .1
    def dqdvdp(X,Q, V, P,S,F, U,t):
        # Vascular parameters
        Eo = .4
        tauo = 2
        tauv = 30
        epsilon = .005
        taus = .8
        tauf = .4
        E = 1 - (1 - Eo)**(1/F)
        alpha = .4
        F = np.log(1 + np.exp(F))

        # Diagonal is implicitly diagonal returning to baseline
        A = -np.eye(100)*2
        # Network self connectivities
        A[:50,:50] = A1
        A[50:,50:] = A3
        UU = U2[t]
        # Autonomous, undriven, unforced, linear dynamics
        dx = -3*X +UU
        # Check condition for which connectivity direction to induce
        if U < 0:
            A[50:,:50] = 1*A2
            U = 1*np.ones(100)*U**2
            C[50:] *= 1
            U = (U)
            dx += (A.dot(X)) +U
        elif U > 0:
            A[:50,50:] = 1*A4
            U = 1*np.ones(100)*U**2
            U = (U)
            C[:50] *= 1
            dx += (A.dot(X)) + U
        # Hemodynamic dynamics
        df = S
        ds = epsilon*X - S/taus - (F-1)/tauf
        dq = F/tauo *(E/Eo - Q/V) + 1/tauv*(F - V**(1/alpha))*Q/V
        dv = 1/tauv*(F - V**(1/alpha))
        dp = 1/tauv*(F - V**(1/alpha))*P/V
        return np.vstack((dq, dv, dp, df, ds,dx))
    # Initial conditions of each voxel
    X[0] = 0*np.ones(100)
    q[0] = 1*np.ones(100)
    p[0] = 6.4*np.ones(100)
    v[0] = 1*np.ones(100)
    s[0] = 0*np.ones(100)
    f[0] = 1*np.ones(100)
    derivf = []
    derivs = []
    derivv = []
    derivp = []
    derivq = []
    derivx = []
    # Adams Bashforth implicit integration FAST!
    for i, (y1_t, y2_t, y3_t, y4_t, y5_t,y6_t) in enumerate(zip(q[:-1],v[:-1],p[:-1], s[:-1], f[:-1],X[:-1])):
        if i < 4:
            k1 = dqdvdp(X[i],q[i], v[i], p[i], s[i], f[i], u[i],i)
            ink2 = (y6_t + k1[5]/2, y1_t + k1[0]/2, y2_t + k1[1]/2, y3_t + k1[2]/2,y4_t + k1[3]/2, y5_t + k1[4]/2,)
            k2 = dqdvdp(*ink2, u[i],i)
            ink3 = (y6_t + k2[5]/2, y1_t + k2[0]/2, y2_t + k2[1]/2, y3_t + k2[2]/2,y4_t + k2[3]/2, y5_t + k2[4]/2,)
            k3 = dqdvdp(*ink3, u[i],i)
            ink4 = (y6_t + k2[5], y1_t + k3[0], y2_t + k3[1], y3_t + k3[2], y4_t + k3[3], y5_t + k3[4],)
            k4 = dqdvdp(*ink4, u[i],i)
            derivq.append(1/6*(k1[0] + k2[0]*2 + 2*k3[0] + k4[0]))
            derivv.append(1/6*(k1[1] + k2[1]*2 + 2*k3[1] + k4[1]))
            derivp.append(1/6*(k1[2] + k2[2]*2 + 2*k3[2] + k4[2]))
            derivf.append(1/6*(k1[3] + k2[3]*2 + 2*k3[3] + k4[3]))
            derivs.append(1/6*(k1[4] + k2[4]*2 + 2*k3[4] + k4[4]))
            derivx.append(1/6*(k1[5] + k2[5]*2 + 2*k3[5] + k4[5]))
            q[i+1] = y1_t + h*derivq[-1]
            v[i+1] = y2_t + h*derivv[-1]
            p[i+1] = y3_t + h*derivp[-1]
            s[i+1] = y4_t + h*derivs[-1]
            f[i+1] = y5_t + h*derivf[-1]
            X[i+1] = y6_t + h*derivx[-1]
        else:
            pds = dqdvdp(X[i],q[i], v[i], p[i], s[i], f[i], u[i],i)
            predx = y1_t + h/24*(55*pds[5] - 59*derivx[-1] + 37*derivx[-2] - 9*derivx[-3])
            predq = y1_t + h/24*(55*pds[0] - 59*derivq[-1] + 37*derivq[-2] - 9*derivq[-3])
            predv = y2_t + h/24*(55*pds[1] - 59*derivv[-1] + 37*derivv[-2] - 9*derivv[-3])
            predp = y3_t + h/24*(55*pds[2] - 59*derivp[-1] + 37*derivp[-2] - 9*derivp[-3])
            predf = y5_t + h/24*(55*pds[3] - 59*derivf[-1] + 37*derivf[-2] - 9*derivf[-3])
            preds = y4_t + h/24*(55*pds[4] - 59*derivs[-1] + 37*derivs[-2] - 9*derivs[-3])
            cds = dqdvdp(predx,predq, predv, predp, preds, predf, u[i],i)
            derivx.append(1/24*(9*cds[5] + 19*pds[5] - 5*derivx[-1] + derivx[-2]))
            derivq.append(1/24*(9*cds[0] + 19*pds[0] - 5*derivq[-1] + derivq[-2]))
            derivv.append(1/24*(9*cds[1] + 19*pds[1] - 5*derivv[-1] + derivv[-2]))
            derivp.append(1/24*(9*cds[2] + 19*pds[2] - 5*derivp[-1] + derivp[-2]))
            derivf.append(1/24*(9*cds[3] + 19*pds[3] - 5*derivf[-1] + derivf[-2]))
            derivs.append(1/24*(9*cds[4] + 19*pds[4] - 5*derivs[-1] + derivs[-2]))
            X[i+1] = y6_t + h*derivx[-1]
            q[i+1] = y1_t + h*derivq[-1]
            v[i+1] = y2_t + h*derivv[-1]
            p[i+1] = y3_t + h*derivp[-1]
            s[i+1] = y4_t + h*derivs[-1]
            f[i+1] = y5_t + h*derivf[-1]
    
    # Standardize and low pass filter
    qcons = (q-q.mean(0))/q.std(0)
    pcons = (p-p.mean(0))/p.std(0)
    b,a = signal.butter(3,.2/(fs/2), "low")
    q = signal.filtfilt(b,a,qcons,axis=0)
    p = signal.filtfilt(b,a,pcons,axis=0)
    np.save("control{0:d}.npy".format(kk),u)
    np.save("qpure{0:d}.npy".format(kk),qpure)
    np.save("xpure{0:d}.npy".format(kk),X)
    return q    

In [ ]:
# Run 100 runs of the simulations
Simuls = np.array(Parallel(n_jobs=100)(delayed(simulate)(k) for k in range(100)))
np.save("Simuls",Simuls)
# Plot Stimuli and Simulations
control = np.load("control0.npy")
plt.subplot(3,1,1)
plt.plot(np.arange(Simuls.shape[1]-500)/100,Simuls[0,500:,:50])
plt.title("Population 1")
plt.subplot(3,1,2)
plt.plot(np.arange(Simuls.shape[1]-500)/100,Simuls[0,500:,50:])
plt.title("Population 2")
plt.subplot(3,1,3)
plt.plot(np.arange(Simuls.shape[1]-500)/100,control[500:])
plt.title("Stimuli")
plt.xlabel("Time (s)")
plt.savefig("SimulPlots.png")

In [ ]:
# Imports for running Analysis
from sklearn.decomposition import PCA
from nilearn.glm.first_level import glover_hrf
import scipy.signal as signal
import scipy.linalg as la
import vgpccmbatch as gp
from torch.multiprocessing import Pool
import torch.multiprocessing as mp

In [ ]:
# Perform deconvolution and get show manifolds
def genManifs(i,j):
    # Downsample to rate of TR=2.5s to make similar to conventional fMRI acquisitions
    Pop = simuls[i,500::250,:]
    Popn = np.zeros((Pop.shape[0]*13,Pop.shape[1]))
    # UpSample to 5Hz before deconvolution
    Popn[::13] = Pop
    b,a = signal.butter(3,1/13,'low')
    Pop = signal.filtfilt(b,a,Popn)
    PPI = np.load("control{0:d}.npy".format(i))[500::250]
    PPIn = np.zeros(len(PPI)*13)
    PPIn[::13] = PPI
    PPIn = np.convolve(np.ones(13),PPIn)
    PPI = PPIn[:len(PPI)*13]
    # Parse which condition we want to test
    if j == 0:
        PPI = 2*((PPI > 0).astype(float)-.5)
    else:
        PPI = 2*((PPI < 0).astype(float)-.5)
    # Standardize signal
    PPI = (PPI - PPI.mean())/PPI.std()
    # Fourier Bases
    hrf = glover_hrf(1*2.5/13,1,50) # HRF    
    bases = [np.ones(len(PPI))] + [np.sin(2*np.pi*(i+1)*np.arange(len(PPI))/len(PPI)) for i in range(1,480)]
    bases += [np.cos(2*np.pi*(i+1)*np.arange(len(PPI))/len(PPI)) for i in range(1,480)]
    bases = np.array(bases).T
    # Convolution matrix
    cm = la.convolution_matrix(hrf,len(PPI))
    basescm = cm[:len(PPI)].dot(bases)
    # Inverting to get Fourier bases coefficients
    B = np.linalg.inv(basescm.T.dot(basescm)).dot(basescm.T).dot(Popn)
    X = bases.dot(B) # Deconvolved signal
    PPI = (X.T*PPI).T # PPI
    PPI = (PPI - PPI.mean(0))/PPI.std(0)
    PI = X
    PI = (PI - PI.mean(0))/PI.std(0) # Deconvolved BOLD
    Pop1 = PI[::5,50:].T # System 1
    Pop2 = PI[::5,:50].T # System 2
    um = PCA()
    um2 = PCA()
    PPIr = PPI[:,50:].T # System 1
    PPIl = PPI[:,:50].T # System 2
    # Dimensionality Reduction    
    um.fit(Pop1.T)
    um2.fit(Pop2.T)
    PPIr1 = um.transform(PPIr.T).T[[0]]
    PPIl2 = um2.transform(PPIl.T).T[[0]]
    m1 = um.transform(Pop1.T).T[[0]]
    m2 = um2.transform(Pop2.T).T[[0]]
    return PPIr1, PPIl2, m1, m2

In [ ]:
# GP Testing Function, return a posteriori covariance
def test(dat,cuda):
    GP = gp.GP()
    return GP.testCoupling(dat[0], dat[1][None,:], dat[2],5, tau=2, cuda=cuda)[1]

def boot(dat):
    pvals = []
    ress = []
    j = dat[0]
    # Which GPU to use in multiprocessing
    cuda = (mp.current_process()._identity[0] - 1)%8
    for i in range(2):
        PPIr1 = dat[1][i]
        PPIl2 = dat[2][i]
        m1 = dat[3][i]
        m2 = dat[4][i]

        args = [[PPIr1, PPIl2, m1],[PPIl2, PPIr1, m2]]
        ret = []
        # Test coupling
        for k in range(len(args)):
            ret+=[test(args[k],cuda)]
        r = ret[0]
        r2 = ret[1]
        res = (r - r2).ravel()
        pvals += [(res[0] > res[1:]).cpu().numpy().mean()]
        ress += [np.array([r.cpu().numpy(),r2.cpu().numpy()])]#es.cpu().numpy()]
    print(pvals,j)
    # Return results
    return np.array(pvals), np.array(ress)

In [ ]:
# Add some noise to the simulations
simuls = Simuls + .1*np.random.randn(*simuls.shape)
PPIr1s PPIl2s, m1s, m2s = zip(*Parallel(n_jobs=250)(delayed(genManifs)(i,j) for i in range(100) for j in range(2)))
print("Got Manifs!")
# Reshape it
PPIr1s = np.array(PPIr1s).reshape(100,2,1,-1)
PPIl2s = np.array(PPIl2s).reshape(100,2,1,-1)
m1s = np.array(m1s).reshape(100,2,1,-1)
m2s = np.array(m2s).reshape(100,2,1,-1)
resss = [] # Results
pvalss = [] # Pvals

mp.set_start_method('spawn',force=True)
with Pool(8) as p:
    args = [[i,PPIr1s[i],PPIl2s[i],m1s[i],m2s[i]] for i in range(100)]
    for i in range(10):
        # Run the CCM analysis 10 times to account for RNG in the initialization of the
        # Coupling measure algorithm
        pvals, ress = zip(*p.map(boot,args))
        pvalss += [pvals]
        resss += [ress]
        np.save("PvalsPPIdeconv.npy", np.array(pvalss).reshape(i+1,100,2))
        np.save("ResPPIdeconv.npy", np.array(resss).reshape(i+1,100,2,2,-1))

In [ ]:
Res = np.median(np.load("ResPPIdeconv.npy"),0)
R = ((Res[:,:,0] - Res[:,:,1])).reshape(*Res.shape[:2],-1)

# Results will be 2 numbers 
#(% Condition 1 Coupling, % Condition 2 Coupling)
# This tests alternate hypothesis System 2 -> System 1
((R[...,[0]] > R[...,1:]).mean(-1) < 0.05).sum(0)/100
#array([0,  .36])
# This test's alternate hypothesis System 1 -> System 2
(1-(R[...,[0]] > R[...,1:]).mean(-1) < 0.05).sum(0)/100
#array([.32,  .01])

# If we change alpha, we see that we gain more statistical power while not sacrificing much specificity.
# Say we change it to 0.35:
((R[...,[0]] > R[...,1:]).mean(-1) < 0.35).sum(0)/100
# array([0.05, 0.91])
(1-(R[...,[0]] > R[...,1:]).mean(-1) < 0.35).sum(0)/100
#array([0.8 , 0.05])
# See, specificity is still 95%, very good, but we increase sensitivity exceptionally, up to at least 80%